In [8]:
import io
import ipywidgets as widgets
import matplotlib.pyplot as plt
import PIL.Image
import numpy as np
import urllib
from skimage.transform import resize
from matplotlib.image import imread
import os
from IPython.display import display
from skimage import io as io_url
import cv2
import numpy as np
from PIL import Image

In [9]:
# Create image widgets
image3_spatial = widgets.Image(format='png', width=500, height=500, description='Spatial')
image3_freq    = widgets.Image(format='png', width=500, height=500, description='Frequency')
sidebyside     = widgets.HBox([image3_spatial, image3_freq])

# Create slider/select widgets
slider_inner   = widgets.FloatSlider(value=0,      min=0, max=1,      step=0.01, description='Inner radius')
slider_outer   = widgets.FloatSlider(value=1.44/2, min=0, max=1.44/2, step=0.01, description='Outer radius')

buf            = io.BytesIO()

orig_img = io_url.imread('https://img2.zergnet.com/2309662_300.jpg')
orig_img = np.mean(orig_img, -1)

x = np.fft.fftfreq(orig_img.shape[0]);
y = np.fft.fftfreq(orig_img.shape[1]);

xv, yv = np.meshgrid(x, y)
xv = np.fft.fftshift(xv)
yv = np.fft.fftshift(yv)

def filter_frequency(orig_img, mask):
    """
    Remove frequency based on the given mask.
    Params:
        orig_img: numpy image
        mask: same shape with orig_img indicating which frequency to hold or remove
    Output:
        f_img: frequency image after applying mask
        img: image after applying mask
    """
    # Compute the 2D Fourier transform of the input image
    f_img = np.fft.fft2(orig_img)
    
    # Apply the mask to the Fourier transform
    f_img_filtered = f_img * mask
    
    # Compute the inverse Fourier transform to obtain the filtered image
    img = np.fft.ifft2(f_img_filtered).real
    
    return f_img_filtered, img

def on_value_change3(change):
    mask = (np.sqrt(xv**2 + yv**2) < slider_outer.value) & \
           (np.sqrt(xv**2 + yv**2) >= slider_inner.value)
    mask = np.float32(mask)

    fimg, img = filter_frequency(orig_img, mask)
    buf.seek(0)
    tmp = PIL.Image.fromarray(255*img/(img.max()+0.0001))
    tmp = tmp.convert('L')
    tmp.save(buf, 'png')
    image3_spatial.value = buf.getvalue()

    buf.seek(0)
    tmp = PIL.Image.fromarray(255*np.log(0.0001*fimg + 1))
    tmp = tmp.convert('L')
    tmp.save(buf, 'png')
    image3_freq.value = buf.getvalue()


slider_inner.observe(on_value_change3, names='value')
slider_outer.observe(on_value_change3, names='value')

on_value_change3(0)

display(sidebyside)
display(slider_inner)
display(slider_outer)

TypeError: Cannot handle this data type: (1, 1), <c16

In [ ]:
def read_img(img_path, img_size=(512, 512)):
  """
    + Đọc ảnh
    + Chuyển thành grayscale
    + Thay đổi kích thước ảnh thành img_size
  """
  img = cv2.imread(img_path, 0)
  img = cv2.resize(img, img_size)
  return img


def create_hybrid_img(img1, img2, r):
    """
    Create hybrid image
    Params:
        img1: numpy image 1
        img2: numpy image 2
        r: radius that defines the filled circle of frequency of image 1.
           This radius determines which frequency components are preserved from img1.
    """
    # Compute the Fourier transform of both images
    f_img1 = np.fft.fft2(img1)
    f_img2 = np.fft.fft2(img2)
    
    # Create a circular mask to preserve low frequencies in img1
    x, y = img1.shape
    xv, yv = np.meshgrid(np.arange(y), np.arange(x))
    center_x, center_y = x // 2, y // 2
    mask = np.sqrt((xv - center_x) ** 2 + (yv - center_y) ** 2) < r * min(center_x, center_y)
    
    # Apply the mask to img1's Fourier transform and (1 - mask) to img2's Fourier transform
    f_img1_low = f_img1 * mask
    f_img2_high = f_img2 * (1 - mask)
    
    # Combine the low-frequency components of img1 with the high-frequency components of img2
    f_hybrid = f_img1_low + f_img2_high
    
    # Compute the inverse Fourier transform to obtain the hybrid image
    hybrid_img = np.fft.ifft2(f_hybrid).real
    
    return hybrid_img


In [10]:
image_1_path = "" # <-- need to change
image_2_path = "" # <-- need to change
img_1 = read_img(image_1_path)
img_2 = read_img(image_2_path)
hybrid_img = create_hybrid_img(img_2, img_1, 14)
fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 15))
axes[0].imshow(img_1, cmap="gray")
axes[0].axis("off")
axes[1].imshow(img_2, cmap="gray")
axes[1].axis("off")
axes[2].imshow(hybrid_img, cmap="gray")
axes[2].axis("off")

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
